## Лекция 3  NER

### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)

In [1]:
def preprocess_with_natasha(text: str) -> str:
    pass

def preprocess_with_deepmipt(text: str) -> str:
    pass

In [ ]:
Буду использовать два готовых экстрактора для имён и адресов, а для дат соберу экстрактор сама и дополню правила

In [ ]:
Это самые базовые правила для извлечения дат, из них я соберу собственный экстрактор.

In [179]:
from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, dictionary, gte, lte, eq
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

In [182]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

class Date(Date):
    @property
    def obj(self):
        from natasha import obj
        return obj.Date(self.year, self.month, self.day)


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}


MONTH_NAME = dictionary(MONTHS)
DAY = and_(
    gte(1),
    lte(31)
)
YEAR = and_(
    gte(1900),
    lte(2100)
)
DATE = rule(
    DAY.interpretation(
        Date.day
    ),
    MONTH_NAME.interpretation(
        Date.month
    ),
    YEAR.interpretation(
        Date.year
    ).optional()
).interpretation(
    Date
)

In [99]:
from natasha.extractors import Extractor
class DatesExtractor(Extractor):
    def __init__(self, morph):
        Extractor.__init__(self, DATE, morph)

In [100]:
from natasha import NamesExtractor, AddrExtractor
from natasha import MorphVocab
morph_vocab = MorphVocab()
name_extractor = NamesExtractor(morph_vocab)
address_extractor = AddrExtractor(morph_vocab)
date_extractor = DatesExtractor(morph_vocab)

In [103]:
def preprocess_with_natasha(text: str) -> str:
    ner_spans = []
    persons = [x for x in name_extractor(text)]
    places = [x for x in address_extractor(text)]
    dates = [x for x in date_extractor(text)]
    ner_spans.extend(dates)
    ner_spans.extend(persons)
    ner_spans.extend(places)
    for entity in ner_spans:
        start, end = entity.start, entity.stop
        text_before_ent = text[:start]
        text_after_ent = text[end:]
        substitute_text = ' ' * (end - start)
        text = f'{text_before_ent}{substitute_text}{text_after_ent}'
    return text

In [ ]:
Попробую удаление NER на тексте

In [107]:
%%time
preprocess_with_natasha('Я, Иванов Иван Иванович прибыл в Россию из Стамбула, Турция. Мне нужна справка?')

CPU times: user 137 ms, sys: 4.05 ms, total: 141 ms
Wall time: 150 ms


'Я,                      прибыл                     , Турция. Мне       справка?'

In [ ]:
В результате есть ложно положительный спан "нужна" и ложно отрицательный "Турция". 
В целом, ок и это без препроцессинга с моей стороны!

In [ ]:
Попробую реализовать удаление NER через deepmipt, но так deeppavlov это то же самое, то буду использовать его.

In [109]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Сейчас будет много ворнингов

In [111]:
%%time
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_rus)

2020-10-21 16:52:28.846 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/elizavetaersova/.deeppavlov/models/ner_rus/word.dict]
2020-10-21 16:52:28.914 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/elizavetaersova/.deeppavlov/models/ner_rus/tag.dict]
2020-10-21 16:52:28.918 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/elizavetaersova/.deeppavlov/models/ner_rus/char.dict]
2020-10-21 16:52:28.922 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 53: [loading fastText embeddings from `/Users/elizavetaersova/.deeppavlov/downloads/embeddings/lenta_lower_100.bin`]
2020-10-21 16:52:36.551 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this 

INFO:tensorflow:Restoring parameters from /Users/elizavetaersova/.deeppavlov/models/ner_rus/model
CPU times: user 5.72 s, sys: 2.98 s, total: 8.7 s
Wall time: 11.8 s


In [116]:
def preprocess_with_deepmipt(text: str) -> str:
    anno = ner_model([text])
    tokens = anno[0][0]
    tags = anno[1][0]
    clean_tokens = []
    for token, tag in zip(tokens, tags):
        if tag == 'O':
            clean_tokens.append(token)
    clean_text = ' '.join(clean_tokens)
    return clean_text

In [117]:
%%time
preprocess_with_deepmipt('Я, Иванов Иван Иванович прибыл в Россию из Стамбула, Турция. Мне нужна справка?')

CPU times: user 17.8 ms, sys: 2.87 ms, total: 20.7 ms
Wall time: 20.6 ms


'Я , прибыл в из , . Мне нужна справка ?'

In [ ]:
Идеальная работа и быстро!

### __Задача 2__:    
На предыдущем занятии вы реализовывали функции поиска ближайших ответов на запросы через TF-IDF и BM25. 
Сравните качество нахождения верного ответа для обоих методов в трех случаях:
- с функцией ```preprocess_with_natasha```
- с функцией ```preprocess_with_deepmipt```
- без препроцессинга

Для измерения качества используйте метрику accuracy. Считаем, что ответ верный, если он входит в топ-1.

In [ ]:
Я понимаю "без препроцессинга" как "без удаления NE".
Для эксперимента возьму сырые тексты, обработаю либо просто препроцессингом, либо препроцессингом и удалением NE

In [134]:
import re
import pandas as pd
from rank_bm25 import BM25Okapi
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [131]:
def preprocess(text):
    text = text.lower()
    text = re.sub('[^а-яё]', ' ', text)
    text = [token for token in text.split() if token not in russian_stopwords]
    text = ' '.join(text)
    return text

In [128]:
X_train = pd.read_csv('X_train.csv')  
X_train =  X_train.rename(columns={'Unnamed: 0': 'question_id'})

In [133]:
%%time
X_train['clean_text'] = X_train.text.apply(lambda x: str(preprocess(x)))
X_train['Natasha_NER_removal'] = X_train.text.apply(lambda x: str(preprocess(preprocess_with_natasha(x))))
X_train['Deepmipt_NER_removal'] = X_train.text.apply(lambda x: str(preprocess(preprocess_with_deepmipt(x))))

CPU times: user 8min 53s, sys: 15.2 s, total: 9min 9s
Wall time: 8min 48s


In [155]:
X_train.head(5)

,question_id,text,clean_text,Natasha_NER_removal,Deepmipt_NER_removal,answer_id
0,354,Добрый день! \n \nПрошу описать ...,добрый день прошу описать порядок действий тре...,прошу описать порядок действий требуемые докум...,добрый день прошу описать порядок действий тре...,308
1,2114,\nДобрый день!\n1.Нужно ли сотруднику прибывше...,добрый день нужно сотруднику прибывшему г тюме...,сотруднику прибывшему самоизоляции дней сдават...,добрый день нужно сотруднику прибывшему г само...,308
2,450,Какие ограничения действуют при поездке в Абха...,какие ограничения действуют поездке абхазию де...,ограничения действуют поездке абхазию карантин...,какие ограничения действуют поездке действуют ...,308
3,1537,"Здравствуйте, подскажите пожалуйста, где можно...",здравствуйте подскажите пожалуйста пройти тест...,подскажите пожалуйста пройти определение пропа...,здравствуйте подскажите пожалуйста пройти тест...,135
4,296,"Добрый день! Подскажите,если я (гражданин Р.Ф....",добрый день подскажите гражданин р ф поеду отп...,подскажите гражданин отпкус наземным транспорт...,добрый день подскажите гражданин р ф поеду отп...,308


In [156]:
%%time
corpus_basic = X_train.clean_text.tolist()
tokenized_basic_corpus = [doc.split() for doc in corpus_basic]
bm25_basic = BM25Okapi(tokenized_basic_corpus)

CPU times: user 53.9 ms, sys: 2.96 ms, total: 56.9 ms
Wall time: 60.8 ms


In [157]:
%%time
corpus_natasha = X_train.Natasha_NER_removal.tolist()
tokenized_corpus_natasha = [doc.split() for doc in corpus_natasha]
bm25_natasha = BM25Okapi(tokenized_corpus_natasha)

CPU times: user 34.9 ms, sys: 1.54 ms, total: 36.4 ms
Wall time: 42.9 ms


In [158]:
%%time
corpus_deepmipt = X_train.Deepmipt_NER_removal.tolist()
tokenized_corpus_deepmipt = [doc.split() for doc in corpus_deepmipt]
bm25_deepmipt = BM25Okapi(tokenized_corpus_deepmipt)

CPU times: user 51.8 ms, sys: 2.67 ms, total: 54.5 ms
Wall time: 66.8 ms


In [159]:
def bm25_search_basic(query):
    query = preprocess(query)
    tokenized_query = query.split()
    answer_text = bm25_basic.get_top_n(tokenized_query, corpus_basic, n=1)
    answer_id = X_train[X_train['clean_text'] == answer_text[0]].iloc[0]['answer_id']
    return answer_id

In [189]:
def bm25_search_natasha_ner(query):
    query = preprocess(preprocess_with_natasha(query))
    tokenized_query = query.split()
    answer_text = bm25_natasha.get_top_n(tokenized_query, corpus_natasha, n=1)
    answer_id = X_train[X_train['Natasha_NER_removal'] == answer_text[0]].iloc[0]['answer_id']
    return answer_id

In [176]:
def bm25_search_deepmipt_ner(query):
    query = preprocess(preprocess_with_deepmipt(query))
    tokenized_query = query.split()
    answer_text = bm25_deepmipt.get_top_n(tokenized_query, corpus_deepmipt, n=1)
    answer_id = X_train[X_train['Deepmipt_NER_removal'] == answer_text[0]].iloc[0]['answer_id']
    return answer_id

In [166]:
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')
X_test = X_test.join(y_test, lsuffix='_caller', rsuffix='_actions')
X_test = X_test.drop(columns=['Unnamed: 0_caller', 'Unnamed: 0_actions'])


In [167]:
X_test

,text,answer_id
0,Добрый вечер! Моя мама сдавала анализ на ПЦР 5...,6
1,Здравствуйте для прохождения на тест карновиру...,308
2,Добрый день.\nВ Новосибирске были сняты ограни...,37
3,\nЗдравствуйте! Я прибыл из Финляндии в Карели...,308
4,Здравствуйте. Я гражданка Российской Федерации...,308
...,...,...
571,"Здравствуйте! В магазине Пятёрочка (г. Калуга,...",1
572,Где сдать экспресс-тест на коронавирус в Сургу...,135
573,У меня большая просьба ответьте на мой вопрос ...,12
574,"Здравствуйте! Подскажите, пожалуйста, в какой ...",37


In [168]:
%%time
X_test['clean_text'] = X_test.text.apply(lambda x: str(preprocess(x)))
X_test['Natasha_NER_removal'] = X_test.text.apply(lambda x: str(preprocess(preprocess_with_natasha(x))))
X_test['Deepmipt_NER_removal'] = X_test.text.apply(lambda x: str(preprocess(preprocess_with_deepmipt(x))))

CPU times: user 4min 35s, sys: 6.5 s, total: 4min 42s
Wall time: 4min 35s


In [171]:
y_test = X_test.answer_id.tolist()
X_test_basic = X_test.clean_text.tolist()
X_test_ner_natasha = X_test.Natasha_NER_removal.tolist()
X_test_ner_deepmipt = X_test.Deepmipt_NER_removal.tolist()

In [ ]:
Без удаления NE

In [173]:
%%time
n_corr = 0
n_queries = len(X_test_basic)
for query, answer in zip(X_test_basic, y_test):
    pred_answer = bm25_search_basic(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.6319444444444444
CPU times: user 10.6 s, sys: 76.5 ms, total: 10.6 s
Wall time: 11 s


In [ ]:
С удалением NE Наташей

In [177]:
%%time
n_corr = 0
n_queries = len(X_test_ner_natasha)
for query, answer in zip(X_test_ner_natasha, y_test):
    pred_answer = bm25_search_natasha_ner(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.6197916666666666
CPU times: user 1min 19s, sys: 658 ms, total: 1min 20s
Wall time: 1min 24s


In [ ]:
С удалением NE Deepmipt

In [178]:
%%time
n_corr = 0
n_queries = len(X_test_ner_deepmipt)
for query, answer in zip(X_test_ner_deepmipt, y_test):
    pred_answer = bm25_search_deepmipt_ner(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.6076388888888888
CPU times: user 27.9 s, sys: 2.24 s, total: 30.1 s
Wall time: 19.7 s


In [ ]:
Удаление NE не улучшает качество поиска 

### __Задача 3__:    
Улучшить правила в natasha. Написать правила, которые ловят даты в следующих примерах и пересчитать статистику из Задачи 2:
- Уехал 8-9 ноября в Сочи
- Уезжаю 5 числа                           
- 20го сентября заболел

Пример можно посмотреть тут: https://github.com/natasha/yargy

In [ ]:
Дополню текущие правила DatesExtractor

In [183]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

class Date(Date):
    @property
    def obj(self):
        from natasha import obj
        return obj.Date(self.year, self.month, self.day)


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь',
    'число'
}


MONTH_NAME = dictionary(MONTHS)


DAY = and_(
    gte(1),
    lte(31)
)


YEAR = and_(
    gte(1900),
    lte(2100)
)


IMP_DATE = rule(
    DAY.interpretation(Date.day),
    eq('го').optional(),
    eq('-').optional(),
    DAY.optional().repeatable().interpretation(Date.day),
    MONTH_NAME.interpretation(Date.month),
    YEAR.interpretation(Date.year).optional()
).interpretation(
    Date
)

In [ ]:
Переопределю экстрактор, чтобы использовать в функции новые правила

In [184]:
from natasha.extractors import Extractor
class ImpDatesExtractor(Extractor):
    def __init__(self, morph):
        Extractor.__init__(self, IMP_DATE, morph)

In [185]:
date_extractor = ImpDatesExtractor(morph_vocab)

In [186]:
%%time
preprocess_with_natasha('8-9 ноября в Сочи')

CPU times: user 29 ms, sys: 2.42 ms, total: 31.4 ms
Wall time: 41.9 ms


'                 '

In [187]:
%%time
preprocess_with_natasha('Уезжаю 5 числа')

CPU times: user 14.1 ms, sys: 356 µs, total: 14.5 ms
Wall time: 15.5 ms


'Уезжаю        '

In [188]:
%%time
preprocess_with_natasha('20го сентября заболел')

CPU times: user 27.3 ms, sys: 1.17 ms, total: 28.5 ms
Wall time: 33.3 ms


'              заболел'

In [ ]:
Во всех случаях новые правила справляются.

In [ ]:
Пересчитаю статистику с использованием новых правил в предобработке запроса

In [190]:
%%time
n_corr = 0
n_queries = len(X_test_ner_natasha)
for query, answer in zip(X_test_ner_natasha, y_test):
    pred_answer = bm25_search_natasha_ner(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.6197916666666666
CPU times: user 1min 20s, sys: 692 ms, total: 1min 20s
Wall time: 1min 24s


In [ ]:
Качество не изменилось